In [1]:
import pandas as pd
import numpy as np

In [2]:
tag_counts = pd.read_csv("./tags_counts_rank.csv")

In [3]:
df = tag_counts[tag_counts['count'] > 1000].sort("avg_rank") # only top tags

In [4]:
df.reset_index(drop=True,inplace=True)

In [5]:
d = pd.read_csv("./X.csv")

In [6]:
from sklearn.manifold import TSNE as tsne
from PIL import Image, ImageDraw

In [7]:
thumb_side = 96 # publication size

## Fill

In [8]:
from shapely.geometry import Point

In [9]:
crit = np.sqrt(2) # adjacency

In [10]:
PRE = "/Users/damoncrockett/Desktop/stpete/viz/tag_fill_crit_alldims_text-ann/"

In [11]:
for tag in list(df.tag):
    tmp = d[d[tag]!=0]
    tmp.reset_index(drop=True,inplace=True)
    tmp = tmp[tmp.columns[(tmp != 0).any()]] # eliminating all zero columns
    dims = str(len(tmp.columns)-1)
    
    num_bin = 3 * int(round( np.sqrt( len(tmp) / np.pi ) )) # circle that fits square, fill 'factor'==3
    
    X = tmp.iloc[:,1:].as_matrix()
    tsne_obj = tsne(n_components=2)
    
    subspace_tsne = pd.DataFrame(tsne_obj.fit_transform(X),columns=["x","y"])
    subspace_tsne['grid_x'] = pd.cut(subspace_tsne['x'],num_bin,labels=False)
    subspace_tsne['grid_y'] = pd.cut(subspace_tsne['y'],num_bin,labels=False)
    subspace_tsne['local_path'] = tmp.local_path
    
    grid_side = num_bin 
    px_w = thumb_side * grid_side
    px_h = thumb_side * grid_side
    
    x,y = range(grid_side) * grid_side, np.repeat(range(grid_side),grid_side)
    grid_list = pd.DataFrame(x,columns=['x'])
    grid_list['y'] = y
    
    point = []
    for i in range(len(grid_list)):
        point.append(Point(grid_list.x.loc[i],grid_list.y.loc[i]))

    grid_list['point'] = point
        
    open_grid = list(grid_list.point)
    canvas = Image.new('RGB',(px_w,px_h),(50,50,50))
    plotted_dict = {}

    plotted = 0
    for i in np.random.permutation(np.arange(np.round(len(subspace_tsne)))):
        im = Image.open(subspace_tsne.local_path.loc[i])
        im.thumbnail((thumb_side,thumb_side),Image.ANTIALIAS)
        x = subspace_tsne.grid_x.loc[i] * thumb_side 
        y = subspace_tsne.grid_y.loc[i] * thumb_side 
        grid_point = Point((subspace_tsne.grid_x.loc[i],subspace_tsne.grid_y.loc[i]))

        if grid_point in open_grid:
            canvas.paste(im,(x,y))
            open_grid.remove(grid_point)
            plotted_dict[grid_point] = subspace_tsne.local_path.loc[i]
            plotted+=1

        elif len(open_grid) == 0:
            break

        else:
            closest_open = min(open_grid,key=lambda x: grid_point.distance(x))
            if grid_point.distance(closest_open) <= crit: # adjacency criterion
                x = int(closest_open.x) * thumb_side
                y = int(closest_open.y) * thumb_side
                canvas.paste(im,(x,y))
                open_grid.remove(closest_open)
                plotted_dict[closest_open] = subspace_tsne.local_path.loc[i]
                plotted+=1

    tagged=0
    for grid_point in plotted_dict:         
        closest_open = min(open_grid,key=lambda x: grid_point.distance(x))
        if grid_point.distance(closest_open) == 1: # strict adjacency          
            local_path = plotted_dict[grid_point]
            tmp = d[d.local_path==local_path]
            tmp = tmp[tmp.columns[(tmp != 0).any()]].iloc[:,1:]
            
            template = Image.new("RGB", (thumb_side, thumb_side), (50,50,50))
            draw = ImageDraw.Draw(template)
            
            h = 5
            while h <= 80:
                for col in list(tmp.columns):
                    draw.text((5,h),col+"_"+str(np.round(tmp[col].iloc[0],2)),fill=(150,150,150)) # muted
                    h+=15
            
            if grid_point.x > closest_open.x:
                draw.rectangle([(90,55),(96,61)],outline="yellow",fill="yellow")
            if grid_point.x < closest_open.x:
                draw.rectangle([(0,55),(6,61)],outline="yellow",fill="yellow")
            if grid_point.y < closest_open.y:
                draw.rectangle([(55,0),(61,6)],outline="yellow",fill="yellow")
            if grid_point.y > closest_open.y:
                draw.rectangle([(55,90),(61,96)],outline="yellow",fill="yellow")
            
            x = int(closest_open.x) * thumb_side
            y = int(closest_open.y) * thumb_side
            canvas.paste(template,(x,y))
            open_grid.remove(closest_open)
            
            tagged+=1
    
    tag_prop = str(np.round(float(tagged)/plotted,2)).lstrip(".")
    prop = str(int(round(plotted*100/len(subspace_tsne))))
    canvas.save(PRE+tag+"_"+str(num_bin)+"_"+prop+"_"+dims+"_"+str(plotted)+"_"+str(tagged)+"_"+tag_prop+".png")